In [ ]:
import numpy as np
import cupy as cp

G = 10

def compute_distances(r):
    # Calculate distances between particles using matrix operations
    delta_r = r[:, None, :] - r[None, :, :]
    distances = cp.sqrt(cp.sum(delta_r**2, axis=-1))
    return distances, delta_r


In [ ]:
def compute_forces(r, m):
    distances, delta_r = compute_distances(r)
    distances = cp.where(cp.eye(len(m)) == 1, 1, distances) + 1e-9  # avoid division by zero
    forces_dir = -delta_r/distances[:, :, None]
    print("m:None")
    print(m[:, None])
    print("mNone:")
    print(m[None,:])    
    # Compute forces using Newton's law of universal gravitation
    forces_mag = G * m[None, :] * m[:, None] / distances**2
    forces = forces_mag[:, :, None] * forces_dir
    return cp.sum(forces, axis=1)

In [ ]:
# Initialize positions, velocities, and masses
r = cp.array([[0, 0, 1], [0, -4, 0], [0, 4, 0]], dtype=cp.float32)  # positions
v = cp.array([[0, 0.02, 0], [2, 0, 0], [-2, 0, 0]], dtype=cp.float32)  # velocities
m = cp.array([1.0, 1.0, 1.0], dtype=cp.float32)  # masses

np_ticks = np.expand_dims(r, axis=0)
cp_ticks = cp.array(np_ticks)

In [ ]:
dt = 0.1
iterations = 1000

for _ in range(iterations):
    forces = compute_forces(r, m)
    v += forces * dt
    r += v * dt
    cp_ticks = cp.append(cp_ticks, cp.expand_dims(r, 0), 0)
    if _ % 20 == 0:
        print(r)

# Transfer the result from GPU to CPU using cp.asnumpy() if necessary
final_positions = cp.asnumpy(r)
print("Final positions:\n", final_positions)

final_velocities = cp.asnumpy(v)
print("Final Velocities:\n", final_velocities)

class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


np_ticks = cp_ticks.get()


# this is data we can work with in python and write to a file
with open("ticks.json", "w") as f:
    f.write(json.dumps(np_ticks, cls=NumpyArrayEncoder))
